In [1]:
import json
import cv2
import os
from os.path import join as pjoin

In [2]:
input_root = 'data/input'
output_root = 'data/output'
detection_dir = pjoin(output_root, 'merge')

In [3]:
gui_name = '0001'
img_android_file = pjoin(input_root, 'A' + gui_name + '.jpg')
data_android_file = pjoin(detection_dir, 'A' + gui_name + '.json')
img_ios_file = pjoin(input_root, 'I' + gui_name + '.png')
data_ios_file = pjoin(detection_dir, 'I' + gui_name + '.json')

In [4]:
img_a = cv2.imread(img_android_file)
img_i = cv2.imread(img_ios_file)
data_a = json.load(open(data_android_file, 'r'))
data_i = json.load(open(data_ios_file, 'r'))

In [6]:
data_a

{'compos': [{'class': 'Compo',
   'height': 23,
   'id': 0,
   'position': {'column_max': 375,
    'column_min': 347,
    'row_max': 67,
    'row_min': 44},
   'width': 28},
  {'class': 'Compo',
   'height': 24,
   'id': 1,
   'position': {'column_max': 426,
    'column_min': 402,
    'row_max': 67,
    'row_min': 43},
   'width': 24},
  {'class': 'Compo',
   'height': 7,
   'id': 2,
   'position': {'column_max': 110,
    'column_min': 99,
    'row_max': 58,
    'row_min': 51},
   'width': 11},
  {'children': [17, 18, 20, 21, 22, 23, 24, 25],
   'class': 'Compo',
   'height': 353,
   'id': 3,
   'position': {'column_max': 449,
    'column_min': 0,
    'row_max': 448,
    'row_min': 95},
   'width': 449},
  {'children': [31, 33, 34, 38],
   'class': 'Compo',
   'height': 117,
   'id': 4,
   'position': {'column_max': 221,
    'column_min': 13,
    'row_max': 691,
    'row_min': 574},
   'width': 208},
  {'children': [32, 35, 36, 37, 39],
   'class': 'Compo',
   'height': 117,
   'id': 5

In [7]:
def resize_bbox(bbox, resize_ratio):
    for key in bbox:
        bbox[key] = int(bbox[key] * resize_ratio)

In [8]:
def clip_compos(org_img, detection_result):
    clips = []
    ratio = org_img.shape[0] / detection_result['img_shape'][0]
    for i, compo in enumerate(detection_result['compos']):
        resize_bbox(compo['position'], ratio)
        position = compo['position']
        clip = org_cp[position['row_min']: position['row_max'], position['column_min']: position['column_max']]
        clip_name = pjoin(clip_dir, compo['class'] + '_' + str(i) + '.jpg')


In [1]:
from match_compos.GUI_pair import GUIPair

In [2]:
gui = GUIPair('0001')

In [3]:
gui.component_detection(False, False, False)

In [4]:
gui.show_detection_result()

No detection result, run component_detection() or load_detection_result() first


In [5]:
gui.load_detection_result()

In [6]:
gui.show_detection_result()